In [41]:
import pandas as pd
import pymysql
import requests
import time
from io import StringIO
from sqlalchemy import create_engine

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [ ]:
# Code Here

import mysql.connector

conn = mysql.connector.connect(host="localhost", user="root", password="", database="temp_db")

invoices = pd.read_sql_query('SELECT * FROM invoices', conn)
order_leads = pd.read_sql_query('SELECT * FROM order_leads', conn)
sales_sql = pd.read_sql_query('SELECT * FROM sales_sql', conn)


with pd.ExcelWriter('SuperMarket.xlsx') as writer:
    invoices.to_excel(writer, sheet_name='invoices')
    order_leads.to_excel(writer, sheet_name='order_leads')
    sales_sql.to_excel(writer, sheet_name='sales_sql')

### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).  

In [31]:
# Code Here

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/adminDivisions"

headers = {
	"X-RapidAPI-Key": "8813f81427msh110825a259e2b2dp1ac084jsnbea18cba81a8",
	"X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET" ,url, headers=headers)

data = response.json()['data']
links = response.json()['links']
metadata = response.json()['metadata']

In [32]:
next_url = links[1]['href']

In [33]:
count = metadata['totalCount']//5

In [ ]:
for i in range(1,11):    
    
    url = "https://wft-geo-db.p.rapidapi.com"
    page = url + next_url
    
    response = requests.request("GET" ,page, headers=headers)
    
    links = response.json()['links']
    
    for item in links:
        if item['rel'] == 'next':
            next_url = item['href']
    
    data.extend(response.json()['data'])
    
    time.sleep(2)

{'links': [{'rel': 'first', 'href': '/v1/geo/adminDivisions?offset=0&limit=5'}, {'rel': 'prev', 'href': '/v1/geo/adminDivisions?offset=50&limit=5'}, {'rel': 'next', 'href': '/v1/geo/adminDivisions?offset=60&limit=5'}, {'rel': 'last', 'href': '/v1/geo/adminDivisions?offset=63365&limit=5'}], 'data': [{'id': 3249002, 'wikiDataId': 'Q1059332', 'name': 'Chaki Wardak', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Maidan Wardak', 'regionCode': 'WAR', 'regionWdId': 'Q183056', 'latitude': 34.108555555, 'longitude': 68.579775, 'population': 0}, {'id': 3292111, 'wikiDataId': 'Q3309261', 'name': 'Chal', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Takhar', 'regionCode': 'TAK', 'regionWdId': 'Q186395', 'latitude': 36.4975, 'longitude': 69.559166666, 'population': 0}, {'id': 3369921, 'wikiDataId': 'Q798508', 'name': 'Chapa Dara', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Kunar', 'regionCode': 'KNR', 'regionWdId': 'Q188147', 'latitude': 34.982777777, 'longitu

In [37]:
len(data)

105

In [39]:
cities = pd.DataFrame(data)

In [42]:
engine = create_engine("mysql+pymysql://root:@localhost/cities")

cities.to_sql("cities", con=engine, if_exists='append')

105

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [104]:
# Code Here

import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [105]:
driver = webdriver.Chrome()
driver.maximize_window()

In [110]:
final_df = pd.DataFrame()

for i in range(1,6):
    
    driver.get("https://www.flipkart.com/search?q=smartphones&page={}".format(i))

    webpage = driver.page_source

    soup = BeautifulSoup(webpage, 'lxml')

    all_divs = soup.find_all('div', class_="tUxRFH")
    
    
    phone_image_url = []
    phone_name = []
    phone_rating = []
    total_rating = []
    total_review = []
    discount_price = []
    actual_price = []

    for phone in all_divs:
        
        try:
            phone_image_url.append(phone.find('img', class_='DByuf4')['src'])
            phone_name.append(phone.find('div', class_='KzDlHZ').text.strip())
            phone_rating.append(phone.find('div', class_='XQDdHH').text.strip())
            total_rating.append(phone.find_all('span', class_='Wphh3N')[0].find_all('span')[0].find_all('span')[0].text.strip())
            total_review.append(phone.find_all('span', class_='Wphh3N')[0].find_all('span')[0].find_all('span')[2].text.strip())
            discount_price.append(phone.find('div', class_='Nx9bqj _4b5DiR').text.strip())
            actual_price.append(phone.find('div', class_='yRaY8j ZYYwLA').text.strip())
            
        except:
            phone_image_url.append(np.nan)
            phone_name.append(np.nan)
            phone_rating.append(np.nan)
            total_rating.append(np.nan)
            total_review.append(np.nan)
            discount_price.append(np.nan)
            actual_price.append(np.nan)

    df = pd.DataFrame({
        'phone_name': phone_name,
        'discount_price': discount_price,
        'actual_price': actual_price,
        'phone_rating': phone_rating,
        'phone_image_url': phone_image_url,
        'total_rating': total_rating,
        'total_review': total_review,
    })
    
    final_df = pd.concat([final_df, df], ignore_index=True)
    
    phone_image_url.clear()
    phone_name.clear()
    phone_rating.clear()
    total_rating.clear()
    total_review.clear()
    discount_price.clear()
    actual_price.clear()
    
    time.sleep(2)

In [111]:
final_df.shape

(120, 7)